In [3]:
#load data from csv
import csv
input_file_path = "/Users/k.abhishek/Documents/customer_clusters/export_all_v2.csv"
def load_data(input_file_path):
    with open(input_file_path, mode ='r') as file:
        csvFile = csv.reader(file)
        data = list(csvFile)
    return data

data = load_data(input_file_path)

In [4]:
countries_in_focus = ["IN", "CN", "US"]
# countries_in_focus = ["IN"]
country_filter = set(countries_in_focus)

#filtering based on country code
# filtered_data = [ d for d in data if d[15] in country_filter]

filtered_data = data.copy()
import random 
random.seed(42)
random.shuffle(filtered_data)

In [5]:
len(filtered_data)

3610594

In [6]:
#find the unique values of legal_cluster_id

legal_cluster_id_set = set([d[1] for d in filtered_data])
print(len(legal_cluster_id_set))

legal_cluster_id_to_data = {}
count_null_legal_cluster_id = 0
for d in filtered_data:
    if d[1] == "null":
        count_null_legal_cluster_id +=1
        continue
    if legal_cluster_id_to_data.get(d[1]) is None:
        legal_cluster_id_to_data[d[1]] = [d]
    else:
        legal_cluster_id_to_data[d[1]].append(d)    
# filtered_data = []
num_entries = 100000
test_corpus = []
comparison_corpus = []
count_entries = 0
for legal_cluster_id, entry in legal_cluster_id_to_data.items():
    # if count_entries > num_entries:
    #     break
    if len(entry) > 1:
        count_entries += len(entry)
        test_corpus.append(entry[0])
        comparison_corpus.extend(entry[1:])
    else:
        comparison_corpus.extend(entry)


2790777


In [7]:
country_wise_corpus = {}
for entry in comparison_corpus:
    country_code = entry[15]
    if country_wise_corpus.get(country_code) is None:
        country_wise_corpus[country_code] = [entry]
    else:
        country_wise_corpus[country_code].append(entry)

In [8]:
import numpy as np
entries_per_legal_cluster_id = [len(v) for k,v in legal_cluster_id_to_data.items()]
# entries_per_legal_cluster_id = [len(v) for k,v in legal_cluster_id_to_data.items() if len(v) > 1]
mean = np.mean(entries_per_legal_cluster_id)
std = np.std(entries_per_legal_cluster_id)
print(mean, std)

1.2266229894480962 1.0011390672114102


In [9]:
sum(entries_per_legal_cluster_id)

3423230

mean and standard deviation for the size of legal_cluster_id
1.5947940644465912 2.2155833789806207

mean and standard deviation for the size of legal_cluster_id after filtering the size of cluster to be > 1
5.20206364596017 4.4183111370491055

In [10]:
count_null_legal_cluster_id

187364

In [11]:
print(len(filtered_data))
print(len(test_corpus))
print(len(comparison_corpus))

3610594
395029
3028201


In [12]:
def extract_tax_numbers(arr):
    unique_tax_numbers = set()
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            unique_tax_numbers.add(arr[i].lower())
    return unique_tax_numbers

def extract_bvd_numbers(arr):
    unique_bvd_numbers = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            unique_bvd_numbers.add(arr[i].lower())
    return unique_bvd_numbers

def match_tax(arr, unique_tax_numbers):
    tax = extract_tax_numbers(arr)
    for tax_num in tax:
        if tax_num in unique_tax_numbers:
            return unique_tax_numbers[tax_num]
    return None

def match_bvd(arr, unique_bvd_numbers):
    bvd = extract_bvd_numbers(arr)
    for bvd_num in bvd:
        if bvd_num in unique_bvd_numbers:
            return unique_bvd_numbers[bvd_num]
    return None

In [13]:
unique_tax_numbers = {}
unique_bvd_numbers = {}
for idx, arr in enumerate(comparison_corpus):
    country_code = arr[15]
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            if unique_bvd_numbers.get(arr[i].lower()):
                unique_bvd_numbers[arr[i].lower()].append(arr)
            else:
                unique_bvd_numbers[arr[i].lower()] = [arr]
            # unique_bvd_numbers[arr[i]] = arr
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            if unique_tax_numbers.get(arr[i].lower()):
                unique_tax_numbers[arr[i].lower()].append(arr)
            else:
                unique_tax_numbers[arr[i].lower()] = [arr]
            # unique_tax_numbers[arr[i]] = arr


In [33]:
import re
import numpy as np
from rapidfuzz import fuzz
from annoy import AnnoyIndex

import re
stop_words = {'llp', 'ltd', 'llc', 'plc', 'inc', 'corp', 'nv', 'group', 'gmbh', 'company', 'corporation', 'limited', 'international', 'import', 'export', 'trading', 'contracting', 'investment', 'co', 'trader', 'traders', 'electronics', 'general', 'decorations', 'industries', 'textile'}
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

# List of common delimiters
def remove_stop_words(words):
    # return " ".join(words)
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

def preprocess_name_2(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)

    # Remove stop words
    clean_name = remove_stop_words(words)
    return clean_name

# Preprocessing function
def preprocess_name_1(name):
    name = name.lower()
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    tokens = name.split()
    tokens.sort()  # Sort tokens for token_sort_ratio
    return ' '.join(tokens)

preprocess_name = preprocess_name_2

In [34]:

def name_to_vector(name, embedding_dim=128):
    vector = np.zeros(embedding_dim)
    tokens = name.split()
    for token in tokens:
        for char in token:
            vector[ord(char) % embedding_dim] += 1  
    return vector

def build_index(corpus, embedding_dim=128, metric='angular', num_trees=25):
    # Preprocess the entire corpus
    # corpus_of_names = ["John Doe", "Jane Smith", "Alice Johnson"]  # Add all million names here
    corpus_of_names = [d[4] for d in corpus]
    preprocessed_names = [preprocess_name(name) for name in corpus_of_names]

    annoy_index = AnnoyIndex(embedding_dim, metric)
    # Convert names to vectors (simple bag-of-words vectorization)


    name_vectors = [name_to_vector(name, embedding_dim) for name in preprocessed_names]

    for i, vector in enumerate(name_vectors):
        # print(i, vector)
        annoy_index.add_item(i, vector)

    annoy_index.build(num_trees) # 50 trees
    return annoy_index, corpus_of_names, preprocessed_names


In [35]:
# # Preprocess the entire corpus
# # corpus_of_names = ["John Doe", "Jane Smith", "Alice Johnson"]  # Add all million names here
# corpus_of_names = [d[4] for d in comparison_corpus]
# preprocessed_names = [preprocess_name(name) for name in corpus_of_names]

# # Build Annoy index
# embedding_dim = 128  # Dimensionality of the embeddings
# annoy_index = AnnoyIndex(embedding_dim, 'angular')

# # Convert names to vectors (simple bag-of-words vectorization)
# def name_to_vector(name):
#     vector = np.zeros(embedding_dim)
#     tokens = name.split()
#     for token in tokens:
#         for char in token:
#             vector[ord(char) % embedding_dim] += 1  
#     return vector

# name_vectors = [name_to_vector(name) for name in preprocessed_names]

# for i, vector in enumerate(name_vectors):
#     # print(i, vector)
#     annoy_index.add_item(i, vector)

# annoy_index.build(50) # 50 trees

In [36]:
# Fuzzy matching score function
function_selection = {
    "ratio": 1,
    "partial_ratio": 0,
    "token_sort_ratio": 1,
    "token_set_ratio": 0,
    "partial_token_sort_ratio": 0,
    "partial_token_set_ratio": 0
}
def combined_fuzzy_score(name1, name2):
    ratio = fuzz.ratio(name1, name2) * function_selection["ratio"]
    partial_ratio = fuzz.partial_ratio(name1, name2) * function_selection["partial_ratio"]
    token_sort_ratio = fuzz.token_sort_ratio(name1, name2)  * function_selection["token_sort_ratio"]
    token_set_ratio = fuzz.token_set_ratio(name1, name2)    * function_selection["token_set_ratio"]
    partial_token_sort_ratio = fuzz.partial_token_sort_ratio(name1, name2) * function_selection["partial_token_sort_ratio"]
    partial_token_set_ratio = fuzz.partial_token_set_ratio(name1, name2) * function_selection["partial_token_set_ratio"]
    # print(f"ratio: {ratio}, partial_ratio: {partial_ratio}, token_sort_ratio: {token_sort_ratio}, token_set_ratio: {token_set_ratio}, partial_token_sort_ratio: {partial_token_sort_ratio}, partial_token_set_ratio: {partial_token_set_ratio}")
    # print(f"Ratios: {ratio}, {partial_ratio}, {token_sort_ratio}, {token_set_ratio}, {partial_token_sort_ratio}, {partial_token_set_ratio}")
    return sum([ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio])/sum(function_selection.values())
    # return max(ratio, partial_ratio, token_sort_ratio, token_set_ratio, partial_token_sort_ratio, partial_token_set_ratio)

In [37]:
embedding_dim = 128
num_trees = 25
index_name_vector_clustered_by_country = {}
preprocessed_names_by_country = {}
corpus_of_names_by_country = {}
for country_code, corpus in country_wise_corpus.items():
    index_name_vector_clustered_by_country[country_code], corpus_of_names_by_country[country_code], preprocessed_names_by_country[country_code] = build_index(corpus, num_trees=num_trees)

In [38]:
# Function to find best match
def find_best_match(input_name, country_code, annoy_index, embedding_dim = 128, num_candidates=15):
    preprocessed_input = preprocess_name(input_name)
    input_vector = name_to_vector(preprocessed_input, embedding_dim)
    # Retrieve nearest neighbors using Annoy
    nearest_neighbors = annoy_index.get_nns_by_vector(input_vector, num_candidates, include_distances=False)
    # print(nearest_neighbors)
    best_match_score = 0
    best_match_name = None
    best_match_idx = None
    preprocessed_names = preprocessed_names_by_country[country_code]
    corpus_of_names = corpus_of_names_by_country[country_code]
    for neighbor_idx in nearest_neighbors:
        candidate = preprocessed_names[neighbor_idx]
        score = combined_fuzzy_score(preprocessed_input, candidate)
        if score > best_match_score:
            best_match_score = score
            best_match_name = corpus_of_names[neighbor_idx]
            best_match_idx = neighbor_idx
    
    return best_match_name, best_match_score, best_match_idx


In [39]:
from tqdm import tqdm
success = 0
failure = 0
not_applicable = 0
result = {}
false_positives = {}
qualified_as_match_threshold = 0
correctly_qualified_as_match_threshold = 0
count_entries_name_matching =0
threshold = 90
for arr in tqdm(test_corpus):
    legal_cluster_id = arr[1].lower()
    matches = {}
    #tax matching
    tax_match = match_tax(arr, unique_tax_numbers)
    if tax_match:
        success += 1
        matches[1] = tax_match
        result[str(arr)] = matches
        continue
    #bvd matching
    bvd_match = match_bvd(arr, unique_bvd_numbers)
    if bvd_match:
        success += 1
        matches[1] = bvd_match
        result[str(arr)] = matches
        continue
    # print("Name matching")
    company_name = arr[4]
    count_entries_name_matching += 1
    country_code = arr[15]
    annoy_index = index_name_vector_clustered_by_country[country_code]
    corpus = country_wise_corpus[country_code]
    best_match_name, best_match_score, best_match_idx = find_best_match(company_name, country_code, annoy_index, embedding_dim=128, num_candidates=15)
    best_matched_entry = None
    if best_match_idx == None:
        not_applicable +=1
        print(arr)
    if best_match_idx and best_match_score > threshold:
        # best_matched_entry = comparison_corpus[best_match_idx]
        best_matched_entry = corpus[best_match_idx]
        # print(best_match_score)
        qualified_as_match_threshold +=1
        if legal_cluster_id == best_matched_entry[1]:
            success +=1
            correctly_qualified_as_match_threshold +=1
        else:
            false_positives[str(arr)] = best_matched_entry
            failure +=1
    else:
        failure +=1
    result[str(arr)] = best_matched_entry
    # print(idx+1, success, failure, success + failure)


  1%|▏         | 5842/395029 [00:02<02:48, 2310.19it/s]

['008e8f8a-95a1-3028-aacd-bd50cff51a83', '589e2c13-205b-38fd-b62b-178929e2690c', 'EG00319745', 'Customer', 'GENERAL TRADING', 'null', 'OCTOBER', 'null', '6', 'null', 'null', 'AL QAHIRA', '11511', 'null', 'null', 'EG', 'Egypt', 'TEL', 'EG', '20', 'null', '34873154', '2017-10-31T11:44:32.289Z', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']


  2%|▏         | 8290/395029 [00:03<02:42, 2376.31it/s]

['2decc138-1155-3783-a9e3-245dc7d1ecff', 'c4b63023-8d57-3772-9eb1-29a42a179670', 'CN01435237', 'Customer', 'INTERNATIONAL LTD', 'null', 'LTD 3/F PERFECT IND BLDG 31', 'null', 'TAI YAU STREET SANPOKONG KLN', 'null', 'null', 'HANGZHOU', '310000', 'ZJ', 'ZHEJIANG', 'CN', 'CHINA', 'null', 'null', 'null', 'null', 'null', '2020-06-20T08:26:04.970Z', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']


  3%|▎         | 10326/395029 [00:04<02:34, 2490.54it/s]

...

 93%|█████████▎| 365623/395029 [02:05<00:10, 2922.41it/s]

['0aad293c-7b98-32df-9ba6-03b8097c638c', 'd2216d5b-f69d-31f1-afe6-aecbf4c6020e', 'EG00133689', 'Customer', 'INTERNATIONAL TRADING COMPANY', '47', 'MUSSADAK STREET', 'null', 'DOKKI', 'null', 'null', 'CAIRO', '11511', 'null', 'null', 'EG', 'Egypt', 'TEL', 'EG', '20', 'null', '34288875', '2016-06-05T17:19:06.679Z', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']


 96%|█████████▌| 378502/395029 [02:09<00:05, 2886.65it/s]

['f617581c-4599-3a89-86ae-b9a46f63c574', 'cf6e6255-ad45-3749-ac80-731f0b605a8c', 'KW003026', 'Customer', 'GENERAL TRADING CO', 'null', 'null', '1079', 'null', 'null', 'null', 'SAFAT', '13011', 'null', 'null', 'KW', 'Kuwait', 'TEL', 'KW', '965', 'null', '22425101', '2016-07-10T19:38:47.472Z', 'KW003026', 'KW0000282352', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null']


100%|██████████| 395029/395029 [02:15<00:00, 2922.31it/s]


In [43]:
print(success, failure)
print(qualified_as_match_threshold, correctly_qualified_as_match_threshold)

307459 87570
183366 170774


319480 75549
207263 182795
0.8087507499449408

0.8819470913766567
24468



311033 83997
205553 177854
0.7873655165430473


In [41]:
success/(success + failure)

0.778320072703523

In [42]:
count_entries_name_matching

258344

In [31]:
precision = correctly_qualified_as_match_threshold/qualified_as_match_threshold
print(precision)
qualified_as_match_threshold - correctly_qualified_as_match_threshold

0.9228976011717418


13792

In [32]:
import os
save_result_filename = "complete_data_" + "false_positives_" + str(embedding_dim) + "_" + str(function_selection['ratio']) + "_" + str(function_selection['partial_ratio']) + "_" + str(function_selection['token_sort_ratio']) + "_" + str(function_selection['token_set_ratio']) + "_" + str(function_selection['partial_token_sort_ratio']) + "_" + str(function_selection['partial_token_set_ratio']) + "_threshold_" + str(threshold) + ".json"
# save_result_filename = "result_name_matching_threshold_85.json"
save_folder_path = "/Users/k.abhishek/Documents/customer_clusters/experiments/approximate_match_approach/results"
save_result_file_path = os.path.join(save_folder_path, save_result_filename)

def save_result(result, save_result_file_path):
    import json
    with open(save_result_file_path, 'w') as f:
        json.dump(result, f)

save_result(false_positives, save_result_file_path)